<a href="https://colab.research.google.com/github/xtenix88/Data-Portfolio/blob/main/Investment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as no
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import time
import os

In [2]:
pip install yfinance

     |████████████████████████████████| 63 kB 391 kB/s 
     |████████████████████████████████| 6.4 MB 19.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
import yfinance as yf

In [4]:
# Start end date defaults
S_DATE = "2017-02-01"
E_DATE = "2023-02-01"
S_DATE_DT = pd.to_datetime(S_DATE)
E_DATE_DT = pd.to_datetime(E_DATE)

In [5]:
stock = yf.Ticker("AAPL")
df = stock.history(period="5y")
df.sort_index(axis = 0.,  ascending=False)


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-02-09,176.050003,176.649994,174.899994,176.279999,66591110.0,0.0000,0.0
2022-02-08,171.729996,175.350006,171.429993,174.830002,74829200.0,0.0000,0.0
2022-02-07,172.860001,173.949997,170.949997,171.660004,77251200.0,0.0000,0.0
2022-02-04,171.679993,174.100006,170.679993,172.389999,82391400.0,0.2200,0.0
2022-02-03,174.257984,176.015754,171.900986,172.679993,89418100.0,0.0000,0.0
...,...,...,...,...,...,...,...
2017-02-15,31.988134,32.165164,31.775696,31.985771,142492400.0,0.0000,0.0
2017-02-14,31.504260,31.886644,31.452331,31.870123,132904800.0,0.0000,0.0
2017-02-13,31.412205,31.586876,31.334311,31.461771,92141600.0,0.0000,0.0


In [6]:
def get_column_from_csv(file, col_name):
    # Try to get the file and if it doesn't exist issue a warning
    try:
        df = pd.read_csv(file)
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df[col_name]

In [7]:
tickers = get_column_from_csv("/content/Wilshire-5000-Stocks.csv", "Ticker")
print(len(tickers))
tickers

5


0    AMZN
1    COST
2     DIS
3    DOCU
4    TSLA
Name: Ticker, dtype: object

In [8]:
def save_to_csv_from_yahoo(folder, ticker):
    stock = yf.Ticker(ticker)
    
    try:
        print("Get Data for : ", ticker)
        # Get historical closing price data
        df = stock.history(period="5y")
    
        # Wait 2 seconds
        time.sleep(2)
        
        # Remove the period for saving the file name
        # Save data to a CSV file
        # File to save to 
        the_file = folder + ticker.replace(".", "_") + '.csv'
        print(the_file, " Saved")
        df.to_csv(the_file)
    except Exception as ex:
        print("Couldn't Get Data for :", ticker)

In [9]:
PATH = "/content/"

In [11]:
for x in range(0, 5):
    save_to_csv_from_yahoo(PATH, tickers[x])
    print("Finished")

Get Data for :  AMZN
/content/AMZN.csv  Saved
Finished
Get Data for :  COST
/content/COST.csv  Saved
Finished
Get Data for :  DIS
/content/DIS.csv  Saved
Finished
Get Data for :  DOCU
/content/DOCU.csv  Saved
Finished
Get Data for :  TSLA
/content/TSLA.csv  Saved
Finished


In [12]:
def get_stock_df_from_csv(tickers):
  try:
    df = pd.read_csv(PATH + tickers + '.csv', index_col=0)
  except FileNotFoundError:
    print("File Doesn't Exist")
  else:
    return df

In [13]:
get_stock_df_from_csv('AMZN')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-02-10,823.820007,828.000000,822.849976,827.460022,2429600,0,0
2017-02-13,831.619995,843.000000,828.549988,836.530029,4172600,0,0
2017-02-14,837.000000,838.309998,831.450012,836.390015,2792400,0,0
2017-02-15,834.000000,842.809998,832.820007,842.700012,2968900,0,0
2017-02-16,841.840027,845.000000,839.380005,844.140015,2714700,0,0
...,...,...,...,...,...,...,...
2022-02-03,2834.750000,2884.949951,2766.659912,2776.909912,11276600,0,0
2022-02-04,3112.129883,3224.000000,3012.159912,3152.790039,12640500,0,0
2022-02-07,3170.399902,3253.820068,3145.000000,3158.709961,5131200,0,0


In [14]:
tickers

0    AMZN
1    COST
2     DIS
3    DOCU
4    TSLA
Name: Ticker, dtype: object

In [15]:
from os import listdir
from os.path import isfile, join

files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers

['AMZN', 'COST', 'TSLA', 'DOCU', 'DIS', 'Wilshire-5000-Stocks']

In [16]:
def add_daily_return_to_df(df):
  df['daily_return'] = (df['Close'] / df['Close'].shift(1)) - 1
  return df

In [17]:
def add_cum_return_to_df(df):
  df['cum_return'] = (1 + df['daily_return']).cumprod()
  return df

In [18]:
def add_bollinger_bands(df):
  df['middle_band'] = df['Close'].rolling(window=20).mean()
  df['upper_band'] = df['middle_band'] + 1.96 * df['Close'].rolling(window=20).std()
  df['lower_band'] = df['middle_band'] - 1.96 * df['Close'].rolling(window=20).std()
  return df

In [19]:
def add_ichimoku(df):
  #Conversion line
  hi_val = df['High'].rolling(window=9).max()
  lo_val = df['Low'].rolling(window=9).min()
  df['Conversion'] = (hi_val+lo_val)/2

  #Base Line
  hi_val2 = df['High'].rolling(window=26).max()
  lo_val2 = df['Low'].rolling(window=26).min()
  df['Baseline'] = (hi_val2+lo_val2)/2

  #Span A
  df['SpanA'] = ((df['Conversion'] + df['Baseline']) / 2)

  #Span B
  hi_val3 = df['High'].rolling(window=52).max()
  lo_val3 = df['Low'].rolling(window=52).min()
  df['SpanB'] = ((hi_val3 + lo_val3)/2).shift(26)

  #Laggine Span
  df['Lagging'] = df['Close'].shift(-26)
  return df



In [20]:
for x in tickers:
  try:
    print("Working on:", x)
    new_df = get_stock_df_from_csv(x)
    new_df = add_daily_return_to_df(new_df)
    new_df = add_cum_return_to_df(new_df)
    new_df = add_bollinger_bands(new_df)
    new_df = add_ichimoku(new_df)
    new_df.to_csv(PATH + x + '.csv')
  except Exception as ex:
    print(ex)

Working on: AMZN
Working on: COST
Working on: TSLA
Working on: DOCU
Working on: DIS
Working on: Wilshire-5000-Stocks
'Close'


In [21]:
df = pd.read_csv('/content/AMZN.csv', index_col=0)

#daily return
df['daily_return'] = (df['Close'] / df['Close'].shift(1)) - 1

#cumulative return
df['cum_return'] = (1 + df['daily_return']).cumprod()

#moving average 20 days
df['middle_band'] = df['Close'].rolling(window=20).mean()
df['upper_band'] = df['middle_band'] + 1.96 * df['Close'].rolling(window=20).std()
df['lower_band'] = df['middle_band'] - 1.96 * df['Close'].rolling(window=20).std()

#Conversion line
hi_val = df['High'].rolling(window=9).max()
lo_val = df['Low'].rolling(window=9).min()
df['Conversion'] = (hi_val+lo_val)/2

#Base Line
hi_val2 = df['High'].rolling(window=26).max()
lo_val2 = df['Low'].rolling(window=26).min()
df['Baseline'] = (hi_val2+lo_val2)/2

#Span A
df['SpanA'] = ((df['Conversion'] + df['Baseline']) / 2)

#Span B
hi_val3 = df['High'].rolling(window=52).max()
lo_val3 = df['Low'].rolling(window=52).min()
df['SpanB'] = ((hi_val3 + lo_val3)/2).shift(26)

#Laggine Span
df['Lagging'] = df['Close'].shift(-26)

#return df
df


,Open,High,Low,Close,Volume,Dividends,Stock Splits,daily_return,cum_return,middle_band,upper_band,lower_band,Conversion,Baseline,SpanA,SpanB,Lagging
Date,,,,,,,,,,,,,,,,,
2017-02-10,823.820007,828.000000,822.849976,827.460022,2429600,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,843.200012
2017-02-13,831.619995,843.000000,828.549988,836.530029,4172600,0,0,0.010961,1.010961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,848.059998
2017-02-14,837.000000,838.309998,831.450012,836.390015,2792400,0,0,-0.000167,1.010792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,847.380005
2017-02-15,834.000000,842.809998,832.820007,842.700012,2968900,0,0,0.007544,1.018418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,845.609985
2017-02-16,841.840027,845.000000,839.380005,844.140015,2714700,0,0,0.001709,1.020158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,846.820007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-03,2834.750000,2884.949951,2766.659912,2776.909912,11276600,0,0,-0.078128,3.355945,3047.985010,3428.513682,2667.456337,2904.270020,3067.52002,2985.895020,3517.734985,NaN
2022-02-04,3112.129883,3224.000000,3012.159912,3152.790039,12640500,0,0,0.135359,3.810202,3042.370508,3413.000004,2671.741011,2985.185059,3067.52002,3026.352539,3517.734985,NaN
2022-02-07,3170.399902,3253.820068,3145.000000,3158.709961,5131200,0,0,0.001878,3.817357,3037.752002,3399.980197,2675.523807,3000.095093,3067.52002,3033.807556,3517.734985,NaN


In [22]:
def plot_with_boll_bands(df, ticker):
  fig = go.Figure()

  candle = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name='Candlestick')

  upper_line = go.Scatter(x=df.index, y=df['upper_band'], 
                          line=dict(color='rgba(250,0,0,0.75)',
                          width=1), name='Upper Band')

  mid_line = go.Scatter(x=df.index, y=df['middle_band'], 
                          line=dict(color='rgba(0,0,250,0.75)',
                          width=1), name='Middle Band')
  
  lower_line = go.Scatter(x=df.index, y=df['lower_band'], 
                          line=dict(color='rgba(0,25,0,0.75)',
                          width=1), name='Lower Band')
  
  fig.add_trace(candle)
  fig.add_trace(upper_line)
  fig.add_trace(mid_line)
  fig.add_trace(lower_line)

  fig.update_xaxes(title='Date', rangeslider_visible=True)
  fig.update_yaxes(title='Price')

  fig.update_layout(title="Bollinger Bands", height=1200, width=1800, showlegend=True)

  fig.show()

In [23]:
import plotly.graph_objs as go

fig = go.Figure()

candle = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name='Candlestick')

upper_line = go.Scatter(x=df.index, y=df['upper_band'], 
                          line=dict(color='rgba(250,0,0,0.75)',
                          width=1), name='Upper Band')

mid_line = go.Scatter(x=df.index, y=df['middle_band'], 
                          line=dict(color='rgba(0,0,250,0.75)',
                          width=1), name='Middle Band')
  
lower_line = go.Scatter(x=df.index, y=df['lower_band'], 
                          line=dict(color='rgba(0,25,0,0.75)',
                          width=1), name='Lower Band')
  
fig.add_trace(candle)
fig.add_trace(upper_line)
fig.add_trace(mid_line)
fig.add_trace(lower_line)

fig.update_xaxes(title='Date', rangeslider_visible=True)
fig.update_yaxes(title='Price')

fig.update_layout(title="Bollinger Bands", height=1200, width=1800, showlegend=True)

fig.show()

In [24]:
costco = get_stock_df_from_csv('COST')
plot_with_boll_bands(costco, 'COST')

In [25]:
def get_fill_color(label):
    if label >= 1:
        return 'rgba(0,250,0,0.4)'
    else:
        return 'rgba(250,0,0,0.4)'

In [26]:
import numpy as np

In [27]:
def get_ichimoku(df):
  candle = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name='Candlestick')

  df1 = df.copy()
  fig = go.Figure()
  df['label'] = np.where(df['SpanA'] > df['SpanB'], 1, 0) #return 1 for green, return 0 for red
  df['group'] = df['label'].ne(df['label'].shift()).cumsum()
  df = df.groupby('group')

  dfs = []
  for name, data in df:
    dfs.append(data)
  for df in dfs:
    fig.add_traces(go.Scatter(x=df.index, y=df.SpanA,
        line=dict(color='rgba(0,0,0,0)')))

    fig.add_traces(go.Scatter(x=df.index, y=df.SpanB,
        line=dict(color='rgba(0,0,0,0)'),
        fill='tonexty',
        fillcolor=get_fill_color(df['label'].iloc[0])))
    
  baseline = go.Scatter(x=df1.index, y=df1['Baseline'],
                        line=dict(color='pink', width=2), name='Baseline')
  
  conversion = go.Scatter(x=df1.index, y=df1['Conversion'],
                          line=dict(color='black', width=1), name='Conversion')
  
  lagging = go.Scatter(x=df1.index, y=df1['Lagging'],
                          line=dict(color='purple', width=2), name='Lagging')
  
  span_a = go.Scatter(x=df1.index, y=df1['SpanA'],
                          line=dict(color='green', width=2, dash='dot'), name='Span A')
  
  span_b = go.Scatter(x=df1.index, y=df1['SpanB'],
                          line=dict(color='red', width=1, dash='dot'), name='Span B')
  
  fig.add_trace(candle)
  fig.add_trace(baseline)
  fig.add_trace(conversion)
  fig.add_trace(lagging)
  fig.add_trace(span_a)
  fig.add_trace(span_b)

  fig.update_layout(height=1200, width=1800, showlegend=True)

  fig.show()



In [28]:
costco = get_stock_df_from_csv('COST')
get_ichimoku(costco)

In [30]:
sec_df = pd.read_csv('/content/big_stock_sectors.csv')

In [34]:
sec_df.head()

,Ticker,Name,Description,Mrkt Cap,Sector
0,A,Agilent Technologies,Life Sciences Tools & Services,48.21B,Healthcare
1,AA,Alcoa,Metals & Mining,11.15B,Materials
2,AAC,Ares Acquisition,Blank Check / SPAC,1.22B,SPAC
3,AACG,ATA Creativity Global,Diversified Consumer Services,33.59M,Discretionary
4,AACI,Armada Acquisition I,Blank Check / SPAC,202.75M,SPAC


In [35]:
sec_df['Sector'].value_counts()

Healthcare                1278
Financials                 823
Information Technology     759
Discretionary              599
SPAC                       555
Industrial                 550
Energy                     308
Materials                  285
Real Estate                281
Industrials                237
Communication              179
Staples                     98
Utilities                   91
Services                    54
Name: Sector, dtype: int64

In [50]:
indus_df = sec_df.loc[sec_df['Sector'] == "Industrial"]
health_df = sec_df.loc[sec_df['Sector'] == "Healthcare"]
it_df = sec_df.loc[sec_df['Sector'] == "Information Technology"]
comm_df = sec_df.loc[sec_df['Sector'] == "Communication"]
staple_df = sec_df.loc[sec_df['Sector'] == "Staples"]
discretion_df = sec_df.loc[sec_df['Sector'] == "Discretionary"]
utility_df = sec_df.loc[sec_df['Sector'] == "Utilities"]
financial_df = sec_df.loc[sec_df['Sector'] == "Financials"]
material_df = sec_df.loc[sec_df['Sector'] == "Materials"]
restate_df = sec_df.loc[sec_df['Sector'] == "Real Estate"]
energy_df = sec_df.loc[sec_df['Sector'] == "Energy"]


In [46]:
industry_df.head()

,Ticker,Name,Description,Mrkt Cap,Sector
17,AATC,Autoscope Technologies,Scientific & Technical Instruments,33.79M,Industrials
88,ACY,AeroCentury,Trading Companies & Distributors,263.68M,Industrials
137,AER,AerCap Holdings,Trading Companies & Distributors,16.02B,Industrials
138,AERC,AeroClean Technologies,Pollution & Treatment Controls,145.15M,Industrials
162,AGCO,AGCO Corporation,Machinery,8.68B,Industrials


In [62]:
sec_df.loc[sec_df['Ticker'] == "DIS"]

,Ticker,Name,Description,Mrkt Cap,Sector
1608,DIS,The Walt Disney Company,Entertainment,281.54B,Communication


In [58]:
sec_df.loc[sec_df['Ticker'] == "COST"]

,Ticker,Name,Description,Mrkt Cap,Sector
1328,COST,Costco Wholesale,Food & Staples Retailing,251.74B,Staples


In [49]:
def get_cum_ret_for_stocks(stock_df):
    tickers = []
    cum_rets = []

    for index, row in stock_df.iterrows():
        df = get_stock_df_from_csv(row['Ticker'])
        if df is None:
            pass
        else:
            tickers.append(row['Ticker'])
            cum = df['cum_return'].iloc[-1]
            cum_rets.append(cum)
    return pd.DataFrame({'Ticker':tickers, 'CUM_RET':cum_rets})

In [63]:

discretion = get_cum_ret_for_stocks(discretion_df)
it = get_cum_ret_for_stocks(it_df)
staple = get_cum_ret_for_stocks(staple_df)
comm = get_cum_ret_for_stocks(comm_df)


File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't

In [64]:
print(discretion.sort_values(by=['CUM_RET'], ascending=False).head(20))
print(it.sort_values(by=['CUM_RET'], ascending=False).head(20))
print(staple.sort_values(by=['CUM_RET'], ascending=False).head(20))
print(comm.sort_values(by=['CUM_RET'], ascending=False).head(20))

  Ticker    CUM_RET
0   TSLA  17.308621
  Ticker   CUM_RET
0   AMZN  3.896007
1   DOCU  3.225774
  Ticker   CUM_RET
0   COST  3.440061
  Ticker   CUM_RET
0    DIS  1.407154


In [65]:
df_tech = get_stock_df_from_csv("AMZN")
get_ichimoku(df_tech)